<center>
<h1>Writing a GitHub webhook in Python: the birth of AsanaBot</h1>
<br>
<h3>13 February 2018
<br>
<br>
Ryan May (@dopplershift)
<br><br>
UCAR/Unidata</h3>
</center>

* Once upon a time I got to spend all of my time writing code
* GitHub issues worked great for tracking my todo list

...and life was happy.

* Now my team spends a lot more time tracking "other" kinds of work
* This has necessitated other solutions

Enter Asana.

![Asana](./Asana-tasks.png)

* Great for collaboratively managing our TODO list...
* ...but now we have to manually enter GitHub issues into Asana

Well that's not going to happen.

Idea: "You know, PyGithub and asana-python exist..."

# Phase 1: ~~Collect Underwear~~ Send Things to Asana

* GitHub Python library can readily give me the issues for a repo

```python
import github
gh_client = github.Github(GITHUB_TOKEN)
org = gh_client.get_organization('Unidata')
repo = org.get_repo('MetPy')
for issue in repo.get_issues()
    sync_task(issue)
```

* Asana can create a task with a custom ID--if you're using OAuth
* Can create new task or update existing

```python
# Body of sync_task
client = get_asana_client()  # OAuth stuff
task_id = ('{0.org}-{0.repository}-{0.number:d}'
           .format(issue))
try:
    workspace = find_workspace(org)
    params = {'external': {'id': task_id},
              'name': ('{0.title} (#{0.number})'
                       .format(issue))}
    client.tasks.create_in_workspace(workspace,
                                     params)
except asana.error.InvalidRequest:# Already exists
    task = client.tasks.find_by_id('external:' + task_id)
    attrs = {'completed': issue.state == 'closed'}
    client.tasks.update(task['id'], attrs)
```

Not shown:
* Details of OAuth
* More pulling apart the issues
* How to decide whether we need a new issue
* Matching org -> workspace, repo -> project
* Matching assignees

Ok, so we can do this part. Now how do we automate it...

# Phase 2: ~~???~~ How do I automate this thing?

## Attempt #1: Heroku

* Easy deployment of Flask apps
* Just add a `requirements.txt` and a `Procfile`
* Can even hook up to GitHub to auto deploy

```python
from flask import Flask, jsonify, request
app = Flask('asana-bot')

@app.route('/hooks/github', methods=['POST'])
def sync():
    try:
        issue = issue_from_json(request.get_json())
        task = sync_issue(issue)  # same as before
    except Exception:
        task = {'message': 'Not an event for me.'}
    return jsonify(task)
```

Not shown:
* Error handling
* Turning json payload into an issue--no need for GitHub API

Of course, this requires *manually* adding a webhook to each repository...

"...but you know, you can make GitHub apps..."

![GitHub App Creation 1](./github-app1.png)

![Github App Creation 2](./github-app2.png)

![Github App Creation 3](./github-app3.png)

Great! Now we have an app where we can easily control what repos are hooked up to the app. So we're done, right?

![Delivery Failure](delivery-failure.png)

![Timeout](timeout.png)

* So it turns out GitHub times out on delivering webhooks after 10 seconds...

* ...and Heroku can take 30s turn your machine back on

"Ok, well I did kind of want to learn about AWS Lambda & API Gateway"

## Attempt #2: Chalice

* Chalice is tool from AWS to simplify deployment of "serverless" apps
* API Gateway
  - AWS gives you an address to point at
  - You can map requests to various routes (e.g. `/my/hook`) to other AWS services
* Lambda
  - Short, on-demand computing
  - Builed at 100ms increments; "permanent" free tier is 400,000 Gb-seconds

* Chalice provides its own (somewhat) flask-like web framework
* Handles
  - Collecting code and dependencies
  - Setting up permissions
  - Creating API Gateway routes

```bash
> chalice new-project helloworld
# edit some code
> chalice deploy
...
Initiating first time deployment...
https://qxea58oupc.execute-api.us-west-2.amazonaws.com/api/

# edit more code
> chalice deploy
```

* Simple and slick
* Requires moving away from flask

"Ok, now let me play with allowing users to authenticate to my app...What do you mean this framework doesn't have a session object?"

* The web framework within Chalice is very lightweight...
* ...which means it doesn't have the full power of flask

## Attempt #3: Zappa

* Zappa is another, unofficial AWS "serverless" tool
* Positives:
  * Works with any WSGI app--like flask
  * Does everything Chalice does
  * Requires less invasive changes to app/repo
  * Has feature to keep lambda "warm"
* Less good:
  * Requires virtual environments--WILL NOT work with conda
  * Default AWS permissions includes permissions for almost every service
  * API Gateway handler seems pretty brute force

![Zappa](zappa.gif)

* You can disable Zappa's handling of permissions
* Having flask means you need it and all of its dependencies in deployment

# Phase 3: Profit!

* I have a working GitHub Application
* Running on AWS API Gateway/Lambda
* Syncing Issues/PRs between GitHub and Asana
* 383 total lines of code, including whitespace and comments

## Heroku vs. AWS

* Heroku
  - Easy to set up and make work
  - Easy deployment from GitHub
  - Free tier goes to sleep after inactivity...and takes some time to come back
  - $7/month for a dyno...that probably would have been the cost effective option
* AWS "serverless"
  - This bot will never expend more than 1% of our free tier
  - Bigger learning curve...
  - ...but I feel like this has been really useful to learn
  - Instead of plugging together libraries, I'm plugging together AWS services

## Chalice vs. Zappa

* Chalice
  - Lightweight (~700kb deploy package)
  - Better handling of AWS (surprise, surprise)
  - Very simple web framework
* Zappa
  - Crude/Brute force AWS set up
    - Ensures success in getting things working
    - Not "production" out of the box
  - Full WSGI support
  - Heavier weight (~6Mb deploy package)

## Future work
* Async using AWS Simple Notification Service
* Configuration of repositories
* GitHub user logins
* Storing various keys and secrets on KMS...rather than my laptop
* Automatic deployment on pushes to GitHub

<center>
<h1>Questions?</h1>
</center>
Resources:
* Slides: https://github.com/dopplershift/Talks/
* AsanaBot: https://github.com/Unidata/AsanaBot
* Chalice: https://github.com/aws/chalice
* Zappa: https://www.zappa.io
* PyGithub: https://github.com/PyGithub/PyGithub
* Asana API: https://github.com/Asana/python-asana